In [1]:
import time
import numpy as np
from cola import torch_fns
from cola.linear_algebra import lazify
from cola.algorithms.preconditioners import AdaNysPrecond
from cola.algorithms.preconditioners import NystromPrecond
from cola.ops import Identity
from cola.algorithms.cg import run_batched_cg
from tests.utils import generate_spectrum, generate_pd_from_diag

## Identity Precond


In [2]:
xnp = torch_fns
dtype = xnp.float32
size = int(2 ** 10)
# coeffs = [0.05, 0.1, 0.15, 0.15, 0.05, 0.5]
coeffs = [0.005 for _ in range(10)]
rank_init, mult = 4, 1.5
case = 0
bounds = (0.2, 0.4, 0.6)
# bounds = (0.05, 0.1, 0.2)
# bounds = (0.2, 0.7, 1.0)
rank = rank_init
tic = time.time()
cg_iters = np.zeros(shape=len(coeffs))
for idx, coeff in enumerate(coeffs):
    print("\n" + "*" * 50)
    print(f"Initial rank {rank:2d}")
    print(f"Coeff: {coeff:1.2f}")
    diag = generate_spectrum(coeff=coeff, scale=1.0, size=size)
    A = lazify(xnp.array(generate_pd_from_diag(diag, dtype=diag.dtype), dtype=dtype))
    if case == 1:
        Nys = NystromPrecond(A=A, rank=100, adjust_mu=False)
    elif case == 2:
        Nys = AdaNysPrecond(A=A, rank=rank, bounds=bounds, mult=mult, adjust_mu=False)
        rank = Nys.rank
        print(f"Selected rank {rank:2d}")
        print(f"Error: {Nys.error:1.5e}")
    else:
        Nys = Identity(A.dtype, A.shape)
    b = xnp.ones(shape=(A.shape[0], 1), dtype=dtype)
    x0 = xnp.zeros_like(b)
    out = run_batched_cg(A, b, x0, max_iters=1000, tolerance=1e-11, preconditioner=Nys, pbar=False)
    cg_iters[idx] = out[-1]['iterations']
    print(f"CG iters: {int(cg_iters[idx]):3d}")
toc = time.time()
print(f"\nTime taken: {toc - tic:2.5f} sec")
print(f"CG Q75 {np.quantile(cg_iters, q=0.75):2.2f}")
print(f"CG Q50 {np.quantile(cg_iters, q=0.50):2.2f}")
print(f"CG Q25 {np.quantile(cg_iters, q=0.25):2.2f}")


**************************************************
Initial rank  4
Coeff: 0.01
CG iters: 796

**************************************************
Initial rank  4
Coeff: 0.01
CG iters: 802

**************************************************
Initial rank  4
Coeff: 0.01
CG iters: 762

**************************************************
Initial rank  4
Coeff: 0.01
CG iters: 811

**************************************************
Initial rank  4
Coeff: 0.01
CG iters: 764

**************************************************
Initial rank  4
Coeff: 0.01
CG iters: 769

**************************************************
Initial rank  4
Coeff: 0.01
CG iters: 809

**************************************************
Initial rank  4
Coeff: 0.01
CG iters: 795

**************************************************
Initial rank  4
Coeff: 0.01
CG iters: 797

**************************************************
Initial rank  4
Coeff: 0.01
CG iters: 803

Time taken: 3.53128 sec
CG Q75 802.75
CG Q50 796.50
CG Q25 

## Fixed Nystrom

In [3]:
xnp = torch_fns
dtype = xnp.float32
size = int(2 ** 10)
# coeffs = [0.05, 0.1, 0.15, 0.15, 0.05, 0.5]
coeffs = [0.005 for _ in range(10)]
rank_init, mult = 4, 1.5
case = 1
bounds = (0.2, 0.4, 0.6)
# bounds = (0.05, 0.1, 0.2)
# bounds = (0.2, 0.7, 1.0)
rank = rank_init
tic = time.time()
cg_iters = np.zeros(shape=len(coeffs))
for idx, coeff in enumerate(coeffs):
    print("\n" + "*" * 50)
    print(f"Initial rank {rank:2d}")
    print(f"Coeff: {coeff:1.2f}")
    diag = generate_spectrum(coeff=coeff, scale=1.0, size=size)
    A = lazify(xnp.array(generate_pd_from_diag(diag, dtype=diag.dtype), dtype=dtype))
    if case == 1:
        Nys = NystromPrecond(A=A, rank=100, adjust_mu=False)
    elif case == 2:
        Nys = AdaNysPrecond(A=A, rank=rank, bounds=bounds, mult=mult, adjust_mu=False)
        rank = Nys.rank
        print(f"Selected rank {rank:2d}")
        print(f"Error: {Nys.error:1.5e}")
    else:
        Nys = Identity(A.dtype, A.shape)
    b = xnp.ones(shape=(A.shape[0], 1), dtype=dtype)
    x0 = xnp.zeros_like(b)
    out = run_batched_cg(A, b, x0, max_iters=1000, tolerance=1e-11, preconditioner=Nys, pbar=False)
    cg_iters[idx] = out[-1]['iterations']
    print(f"CG iters: {int(cg_iters[idx]):3d}")
toc = time.time()
print(f"\nTime taken: {toc - tic:2.5f} sec")
print(f"CG Q75 {np.quantile(cg_iters, q=0.75):2.2f}")
print(f"CG Q50 {np.quantile(cg_iters, q=0.50):2.2f}")
print(f"CG Q25 {np.quantile(cg_iters, q=0.25):2.2f}")


**************************************************
Initial rank  4
Coeff: 0.01
CG iters: 503

**************************************************
Initial rank  4
Coeff: 0.01
CG iters: 505

**************************************************
Initial rank  4
Coeff: 0.01
CG iters: 479

**************************************************
Initial rank  4
Coeff: 0.01
CG iters: 483

**************************************************
Initial rank  4
Coeff: 0.01
CG iters: 481

**************************************************
Initial rank  4
Coeff: 0.01
CG iters: 499

**************************************************
Initial rank  4
Coeff: 0.01
CG iters: 481

**************************************************
Initial rank  4
Coeff: 0.01
CG iters: 494

**************************************************
Initial rank  4
Coeff: 0.01
CG iters: 496

**************************************************
Initial rank  4
Coeff: 0.01
CG iters: 507

Time taken: 2.94936 sec
CG Q75 502.00
CG Q50 495.00
CG Q25 

## Adaptive Nystrom

In [4]:
xnp = torch_fns
dtype = xnp.float32
size = int(2 ** 10)
# coeffs = [0.05, 0.1, 0.15, 0.15, 0.05, 0.5]
coeffs = [0.005 for _ in range(10)]
rank_init, mult = 4, 1.5
case = 2
bounds = (0.2, 0.4, 0.6)
# bounds = (0.05, 0.1, 0.2)
# bounds = (0.2, 0.7, 1.0)
rank = rank_init
tic = time.time()
cg_iters = np.zeros(shape=len(coeffs))
for idx, coeff in enumerate(coeffs):
    print("\n" + "*" * 50)
    print(f"Initial rank {rank:2d}")
    print(f"Coeff: {coeff:1.2f}")
    diag = generate_spectrum(coeff=coeff, scale=1.0, size=size)
    A = lazify(xnp.array(generate_pd_from_diag(diag, dtype=diag.dtype), dtype=dtype))
    if case == 1:
        Nys = NystromPrecond(A=A, rank=100, adjust_mu=False)
    elif case == 2:
        Nys = AdaNysPrecond(A=A, rank=rank, bounds=bounds, mult=mult, adjust_mu=False)
        rank = Nys.rank
        print(f"Selected rank {rank:2d}")
        print(f"Error: {Nys.error:1.5e}")
    else:
        Nys = Identity(A.dtype, A.shape)
    b = xnp.ones(shape=(A.shape[0], 1), dtype=dtype)
    x0 = xnp.zeros_like(b)
    out = run_batched_cg(A, b, x0, max_iters=1000, tolerance=1e-11, preconditioner=Nys, pbar=False)
    cg_iters[idx] = out[-1]['iterations']
    print(f"CG iters: {int(cg_iters[idx]):3d}")
toc = time.time()
print(f"\nTime taken: {toc - tic:2.5f} sec")
print(f"CG Q75 {np.quantile(cg_iters, q=0.75):2.2f}")
print(f"CG Q50 {np.quantile(cg_iters, q=0.50):2.2f}")
print(f"CG Q25 {np.quantile(cg_iters, q=0.25):2.2f}")


**************************************************
Initial rank  4
Coeff: 0.01
Selected rank  9
Error: 4.64757e-01
CG iters: 610

**************************************************
Initial rank  9
Coeff: 0.01
Selected rank  9
Error: 2.94144e-01
CG iters: 594

**************************************************
Initial rank  9
Coeff: 0.01
Selected rank  9
Error: 3.64415e-01
CG iters: 598

**************************************************
Initial rank  9
Coeff: 0.01
Selected rank  9
Error: 2.49317e-01
CG iters: 600

**************************************************
Initial rank  9
Coeff: 0.01
Selected rank  9
Error: 3.21118e-01
CG iters: 566

**************************************************
Initial rank  9
Coeff: 0.01
Selected rank 14
Error: 4.61459e-01
CG iters: 607

**************************************************
Initial rank 14
Coeff: 0.01
Selected rank  9
Error: 1.99728e-01
CG iters: 586

**************************************************
Initial rank  9
Coeff: 0.01
Selected r